# Hard spheres in 2D

In [3]:
import numpy as np
import plotly.offline as py
from plotly.graph_objs import *

py.init_notebook_mode(connected=True)

%load_ext cython

In [4]:
DISK_RADIUS = 1

def distances(x, config, x_pos):
    """Return the array of distances of x from any other disk in config."""
    dist = np.sqrt(np.sum(np.power(config - x, 2), axis=1))
    dist[x_pos] = 10*DISK_RADIUS
    return dist

def neighbours(x, config, x_pos):
    """Return the list of the particles position of the disks that
    have distance less than 2.8*disk radius from the disk in position x."""

    dist_min = distances(x, config, x_pos)

    position = (x < 2*DISK_RADIUS) -1*(x > system_size - 2*DISK_RADIUS)


    if np.sum(np.abs(position)):
        clone = x + system_size*position
        dist = distances(clone, config, x_pos)
        dist_min = np.minimum(dist_min, dist)
        k = np.where(dist == dist_min)


    if np.sum(np.abs(position)) == 2:
        clone1 = x + system_size*np.array([0, position[1]])
        clone2 = x + system_size*np.array([position[0], 0])
        dist1 = distances(clone1, config, x_pos)
        dist2 = distances(clone2, config, x_pos)
        dist_min = np.minimum(dist_min, np.minimum(dist1, dist2))


    dist_min[x_pos] = 10*DISK_RADIUS

    neighbours = np.where(dist_min < 2.8*DISK_RADIUS)
    pos = np.zeros((len(neighbours[0]),2))

    if np.sum(np.abs(position)):

        for i in range(len(neighbours[0])):

            if dist_min[neighbours[0][i]] == dist[neighbours[0][i]]:
                pos[i] = position
            elif np.sum(np.abs(position)) == 2:
                if dist_min[neighbours[0][i]] == dist1[neighbours[0][i]]:
                    pos[i] = np.array([0, position[1]])
                elif dist_min[neighbours[0][i]] == dist2[neighbours[0][i]]:
                    pos[i] = np.array([position[0], 0])

    return neighbours, pos


def psi_6(i, config, neighbour_list, pos, system_size):

    z = config[i] + pos*system_size - config[neighbour_list]
    if len(z) > 1:
        angles = np.arctan(z[:,1]/z[:,0])
    else:
        angles = np.arctan(z[1]/z[0])
    return np.sum(np.exp(6j*angles)) / len(neighbour_list[0])


def collision(x, config, x_pos=-1):
    """Return True if the disk in position x collide with any other in config.
    optional parameter:
    - x_pos: the position of x in config (if present)
    """

    for i in range(len(config)):
        if i != x_pos and np.sqrt(np.sum((x-config[i])**2)) < 2*DISK_RADIUS:
            return True

    return False


def random_position(size):
    """Random position in a square (uniform sampling)."""
    return np.random.uniform(0, size, 2)


def configuration_direct_sampling(N, system_size):
    """Direct uniform extraction of N non-overlapping disks in a square."""
    config = []


    for i in range(N):
        x = random_position(system_size)
        y = [0, system_size, -system_size]
        x_pbc_clones = [ x + k for k in np.transpose([np.tile(y, len(y)), np.repeat(y, len(y))])]

        for j in x_pbc_clones:
            if collision(np.array(j) , np.array(config)):
                rejected_counter[0] += 1
                return configuration_direct_sampling(N, system_size)

        config += [x]

    return np.array(config)



def generate_square_lattice(system_size, num_disks):
    """Generates a square lattice with a given number of disks."""
    ss_disks = np.sqrt(num_disks)
    step = system_size / ss_disks
    x = [ step*(.5 + i) for i in range(int(ss_disks))]
    return np.transpose([np.tile(x, len(x)), np.repeat(x, len(x))])


def disk_shape(xc, yc, radius):
    x0 = xc - radius
    y0 = yc - radius
    x1 = xc + radius
    y1 = yc + radius

    return {
        'type': 'circle',
        'xref': 'x',
        'yref': 'y',
        'fillcolor': '#1F77B4',
        'line': dict(width=0),
        'x0': x0,
        'y0': y0,
        'x1': x1,
        'y1': y1,
    }

def plot_config(x, y, system_size):
    """Plot the configuration of the system."""
    trace = Scatter(
        x=x,
        y=y,
        mode='markers'
    )

    shapes = [{'type': 'square', 'x0': 0, 'y0': 0, 'x1': system_size, 'y1': system_size, 'xref':'x', 'yref':'y'}]
    
    for i in range(len(x)):
        shapes.append(disk_shape(x[i], y[i], DISK_RADIUS))
    
    lyt = Layout(
        yaxis=dict(scaleanchor="x", showgrid=False),
        xaxis=dict(showgrid=False),
        shapes=shapes,
        hovermode="closest"
    )
    
    

    fig = dict(data=[trace], layout=lyt)

    py.iplot(fig)



def system_size(num_disks, density, disk_area):
    """Compute the system size to have that num_disks and density."""
    return np.sqrt(num_disks * disk_area / density)


def density(num_disks, syst_size):
    """Compute the density of disks (the fraction of area occupied by disks)."""
    return np.pi * num_disks / syst_size**2


def clone_collision(x, config, system_size):
    """Check if the disk is in the area in which is periodic clones can
       undergo collisions."""

    position = (x < 2*DISK_RADIUS) -1*(x > system_size - 2*DISK_RADIUS)

    if np.sum(np.abs(position)):
        clone = x + system_size*position
        if collision(clone, config):
            return True

    if np.sum(np.abs(position)) == 2:
        clone1 = x + system_size*np.array([0, position[1]])
        clone2 = x + system_size*np.array([position[0], 0])
        if collision(clone1, config) or collision(clone2, config):
            return True

    return False


def reposition_disk(x, system_size):
    """Reposition the disk inside the square if it's out."""
    out = np.array(-1*(x > system_size) + (x < 0))

    if np.sum(np.abs(out)):
        x += system_size * out

    return x


def move_disk(config, num_disks, step_size, system_size):
    """Try to move a random disk of the current config."""
    # move proposal
    disk = np.random.randint(num_disks)
    step = np.random.uniform(-step_size, step_size, 2)
    final_position = config[disk] + step

    # check if there are collision (considering also the PBCs)
    if (collision(final_position, config, disk)
        or clone_collision(final_position, config, system_size)):
        return config

    # reposition the disk if it is out
    final_position = reposition_disk(final_position, system_size)

    config[disk] = final_position

    return config


In [5]:
def _collision(x, config, x_pos=None):
    """Return True if the disk in position x collide with any other in config."""
    distances = np.sum(np.power(config - x, 2), axis=1)
    if x_pos is not None:
        distances[x_pos] = (3*DISK_RADIUS)**2
    
    return np.any(distances < (2*DISK_RADIUS)**2)


def _clone_collision(x, config, system_size):
    """Check if the disk is in the area in which is periodic clones can
       undergo collisions."""

    position = (x < 2*DISK_RADIUS) -1*(x > system_size - 2*DISK_RADIUS)

    if np.sum(np.abs(position)):
        clone = x + system_size*position
        if _collision(clone, config):
            return True

    if np.sum(np.abs(position)) == 2:
        clone1 = x + system_size*np.array([0, position[1]])
        clone2 = x + system_size*np.array([position[0], 0])
        if _collision(clone1, config) or _collision(clone2, config):
            return True

    return False

def _move_disk(config, num_disks, step_size, system_size):
    """Try to move a random disk of the current config."""
    # move proposal
    disk = np.random.randint(num_disks)
    step = np.random.uniform(-step_size, step_size, 2)
    final_position = config[disk] + step

    # check if there are collision (considering also the PBCs)
    if (_collision(final_position, config, disk)
        or _clone_collision(final_position, config, system_size)):
        return config

    # reposition the disk if it is out
    final_position = reposition_disk(final_position, system_size)

    config[disk] = final_position

    return config

In [ ]:
%%cython

import numpy as np
cimport numpy as np
from libc.stdlib cimport rand, RAND_MAX

cdef double DISK_RADIUS = 1.

cdef bint c_collision(np.ndarray x, np.ndarray config, int x_pos=-1):
    """Return True if the disk in position x collide with any other in config."""
    cdef np.ndarray distances = np.sum(np.power(config - x, 2), axis=1)
    
    if x_pos >= 0:
        distances[x_pos] = (3*DISK_RADIUS)**2
        
    return np.any(distances < (2*DISK_RADIUS)**2)


cdef bint c_clone_collision(np.ndarray x, np.ndarray config, double system_size):
    """Check if the disk is in the area in which his periodic clones can
    undergo collisions."""

    cdef np.ndarray position = (x < 2*DISK_RADIUS) -1*(x > system_size - 2*DISK_RADIUS)
    cdef np.ndarray clone, clone1, clone2
    
    if np.sum(np.abs(position)):
        clone = x + system_size*position
        if c_collision(clone, config):
            return True

    if np.sum(np.abs(position)) == 2:
        clone1 = x + system_size*np.array([0, position[1]])
        clone2 = x + system_size*np.array([position[0], 0])
        if c_collision(clone1, config) or c_collision(clone2, config):
            return True

    return False

cdef np.ndarray c_reposition_disk(np.ndarray x, double system_size):
    """Reposition the disk inside the square if it's out."""
    cdef np.ndarray out = np.array(-1*(x > system_size) + (x < 0))

    if np.sum(np.abs(out)):
        x += system_size * out

    return x


cpdef np.ndarray c_move_disk(np.ndarray config, int num_disks, double step_size, double system_size):
    """Try to move a random disk of the current config."""
    # move proposal
    cdef int disk = rand() % num_disks
    cdef np.ndarray step = np.array([rand()/RAND_MAX, rand()/RAND_MAX])*2*step_size - step_size
    cdef np.ndarray final_position = config[disk] + step

    # check if there are collision (considering also the PBCs)
    if (c_collision(final_position, config, disk)
        or c_clone_collision(final_position, config, system_size)):
        return config

    # reposition the disk if it is out
    final_position = c_reposition_disk(final_position, system_size)

    config[disk] = final_position

    return config

We first try to implement a direct sampling strategy. We expect that reaching large density will be practically impossible, since the probability of obtaining two overlapping disk in the extracted configuration will be very high (the acceptance will tend to zero).

In [8]:
rejected_counter = [0]
num_disks = 43
syst_size = 40
c = configuration_direct_sampling(num_disks, syst_size)

plot_config(c[:,0], c[:,1], syst_size)
print("For a density of disks of {0:4f} we get an acceptance ratio of 1/{1}". \
      format(density(num_disks, syst_size), rejected_counter[0] + 1))

For a density of disks of 0.084430 we get an acceptance ratio of 1/992


As we can see even for small density the computational cost of obtaining a single configuration is very high. Also the presence of the periodic boundary condictions (PBCs) increases the computational cost. In fact, for each step, we have to check if the new disk extracted collides with any other (due to the PBCs, if a disk go beyond the square border,it can collide with the disks near the other side of the square). In this first approach we have controlled for each step if any of the particles duplicate (generated by the PBCs) collided with any other particle. In the following we'll do better, doing this check only when the disk is sufficiently near to the border (less than 2 times its radius).

## Markov chain Monte Carlo with square lattice initial condition


In [ ]:
NUM_STEPS = 10
num_disks = 16**2
disk_area = np.pi * DISK_RADIUS**2

for density in [0.5, 0.55, 0.6, 0.65, 0.72]:
    
    print("Plot for density {}, after {} steps".format(density, NUM_STEPS))
    
    syst_size = system_size(num_disks, density, disk_area)
    config = generate_square_lattice(syst_size, num_disks)

    for i in range(NUM_STEPS):
        config = c_move_disk(config, num_disks, 0.5*DISK_RADIUS, syst_size)
        
    plot_config(config[:, 0], config[:, 1], syst_size)
            
    for i in range(num_disks):
        neighbour_list, positions = neighbours(config[i], config, i)
    
        psi[i] = psi_6(i, config, neighbour_list, positions, syst_size)
    
    psi_tot = np.sum(psi)

    print("Psi_6 = {}".format(psi_tot))

#config after NUM_STEPS steps
# plot_config(config[:,0], config[:,1], syst_size)


Ater some trial we get an acceptance of about 0.5 for:
- a step size of 0.65*DISK_RADIUS for density=0.5
- a step size 0.11*DISK_RADIUS for density=0.72
for the a density of 0.5 nothing particular happens: the system gets disordered and then remains disordered, behaving like a liquid in some way.
When the density increases we can see that after long enough domains of ordered (aligned) particles arise in the system (that behaves like a solid).

Measuring randomness of 2d patterns: https://blogs.sas.com/content/iml/2011/02/04/divide-and-count-a-test-for-spatial-randomness.html

In the following we just show how, using in a clever way numpy and applying others small tricks, we can greatly increase the preformances of our program (the definitive version is 20 times faster the one in which we had blindly used for loops)

In [ ]:
import timeit

num_disks = 16**2
disk_area = np.pi * DISK_RADIUS**2
syst_size = system_size(num_disks, 0.5, disk_area)
config = generate_square_lattice(syst_size, num_disks)

def test_move_legacy():
    move_disk(config, num_disks, 0.5*DISK_RADIUS, syst_size)

def test_move():
    _move_disk(config, num_disks, 0.5*DISK_RADIUS, syst_size)

def test_cmove():
    c_move_disk(config, num_disks, 0.5*DISK_RADIUS, syst_size)

    
#config[0, 0] = 0

#_clone_collision(config[0], config, syst_size)
#plot_config(config[:, 0], config[:, 1], syst_size)

    
print("LEG", timeit.timeit(test_move_legacy, number=1000))
print("UPD", timeit.timeit(test_move, number=1000))
print("C", timeit.timeit(test_cmove, number=1000))